<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [7]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag

In [8]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [9]:
def evaluering_resultater(detailed_objective):
  rmse_test = detailed_objective(study.best_trial)[0]
  r2_test = detailed_objective(study.best_trial)[1]
  mae_test = detailed_objective(study.best_trial)[2]
  mape_test = detailed_objective(study.best_trial)[3]

  rmse_trening = detailed_objective(study.best_trial)[4]
  r2_trening = detailed_objective(study.best_trial)[5]
  mae_trening = detailed_objective(study.best_trial)[6]
  mape_trening = detailed_objective(study.best_trial)[7]

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [10]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

# Importering av relevant data

In [11]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [12]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [12]:
collagen_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29136 entries, 2022-10-31 17:37:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29136 non-null  int64  
 1   EnzymeType_A2  29136 non-null  int64  
 2   EnzymeType_B   29136 non-null  int64  
 3   EnzymeType_C   29136 non-null  int64  
 4   EnzymeType_D   29136 non-null  int64  
 5   EnzymeType_E   29136 non-null  int64  
 6   RawMatPercent  29136 non-null  float64
 7   NIRfat         29136 non-null  float64
 8   NIRash         29136 non-null  float64
 9   NIRwater       29136 non-null  float64
 10  TT08           29136 non-null  float64
 11  TT20           29136 non-null  float64
 12  TT12           29136 non-null  float64
 13  Collagen       89 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [13]:
collagen_data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18568 entries, 2022-11-02 00:00:00 to 2023-06-14 01:06:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18568 non-null  int64  
 1   EnzymeType_A2  18568 non-null  int64  
 2   EnzymeType_B   18568 non-null  int64  
 3   EnzymeType_C   18568 non-null  int64  
 4   EnzymeType_D   18568 non-null  int64  
 5   EnzymeType_E   18568 non-null  int64  
 6   RawMatPercent  18568 non-null  float64
 7   NIRfat         18568 non-null  float64
 8   NIRash         18568 non-null  float64
 9   NIRwater       18568 non-null  float64
 10  TT08           18568 non-null  float64
 11  TT20           18568 non-null  float64
 12  TT12           18568 non-null  float64
 13  Collagen       31 non-null     float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


# NIR målinger inkludert

## Alternativ 1: Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

### Alternativ 1: Et testsett

In [14]:
# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert, collagen_test, _, _, _, _\
=trening_testsett_oppdeling_enzym(rå_data,
                                  collagen_data,
                                  test_andel=test_andel)

# Deler datasettene i verdier for forklaringsvariabler og responsvariabler
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparamter optimalisering med Optuna

In [17]:
def objective(trial):

  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere,
                                       random_state=random_seed))
  ])

  # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse
  splitt = 2
  gjentagelser = 2

  rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                 n_repeats=gjentagelser,
                                 random_state=random_seed)

  rmse_validering_resultat = []

  for train, test in rskf.split(collagen_trening_markert,
                                collagen_enzymtyper_markert):

    X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
    X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]

    y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]

    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  rmse_validering_snitt = mean(rmse_validering_resultat)

  return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

Evaluering av beste modell

In [18]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

rfr_pipeline = Pipeline([
    ('skalerer', StandardScaler()),
    ('modell', RandomForestRegressor(**beste_parametere,
                                     random_state=random_seed))
    ])

# Trener modell for videre evaluering
rfr_pipeline.fit(X_trening_markert, y_trening_markert)
y_test_prediksjon = rfr_pipeline.predict(X_test)
y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                       y_test_prediksjon,
                       y_trening_markert,
                       y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

In [19]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,4.311254,0.465149,3.625746,0.190874,2.719353,0.708171,2.158831,0.105589,"{'n_estimators': 191, 'max_depth': 2}"


### Alternativ 1: Flere testandeler

In [ ]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:

  collagen_trening_markert, collagen_enzymtyper_markert, collagen_test,\
  _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                    collagen_data,
                                    test_andel=test_andel)

  X_trening = collagen_trening_markert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 10)
    }
    rfr_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere,
                                         random_state=random_seed))
    ])

    # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse
    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(collagen_trening_markert,
                                 collagen_enzymtyper_markert):

      X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                      random_state=random_seed))
      ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

Test andel: 0.1
Test andel: 0.2


In [ ]:
collagen_rfr_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,4.431145,0.487955,3.908301,0.177555,1.227786,0.940510,0.953507,0.046323,"{'n_estimators': 120, 'max_depth': 7}"
1,0.2,4.003539,0.538774,3.479749,0.176571,1.232411,0.940061,0.955381,0.046436,"{'n_estimators': 119, 'max_depth': 7}"


### Alternativ 1: Flere treningsandeler

In [12]:
collagen_rfr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

# Setter fast testandel
test_andel = 0.1

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert,\
collagen_test, _, collagen_trening_umarkert,\
_, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                       collagen_data,
                                                       test_andel=test_andel)

# Treningssett
X_test = collagen_test.iloc[:, :-1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

In [22]:
# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_trening_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         collagen_trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = collagen_trening_markert
    enzymtyper_trening_markert = collagen_enzymtyper_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 2
    gjentagelser = 2

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                  n_repeats=gjentagelser,
                                  random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_trening_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_enzym.loc[len(collagen_rfr_resultater_enzym)] = resultater

In [23]:
collagen_rfr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.8,4.095496,0.562590,3.193297,0.148836,1.105432,0.954453,0.899073,0.042896,"{'n_estimators': 166, 'max_depth': 10}"
1,0.9,4.234371,0.532422,3.538495,0.164163,1.253540,0.939631,1.037622,0.049491,"{'n_estimators': 167, 'max_depth': 6}"
2,1.0,4.406019,0.493746,3.745152,0.174643,1.791566,0.877468,1.470016,0.070043,"{'n_estimators': 134, 'max_depth': 4}"


**KNeighborsRegressor**

In [32]:
# Treningsandeler
trenings_andeler = [0.8, 0.85, 0.9, 0.95, 1.0]

for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:
    print("i")
    ubenyttet_andel = (1.0-trenings_andel)
    treningssett_markert, enzymtyper_trening_markert,\
    _, _, treningssett_umarkert, _, treningssett\
    = trening_testsett_oppdeling_enzym(rå_data,
                                       collagen_trening,
                                       test_andel=ubenyttet_andel,
                                       random_seed=random_seed)

  else:
    print("j")
    treningssett_markert = collagen_trening_markert
    enzymtyper_trening_markert = collagen_enzymtyper_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])

    splitt = 2
    gjentagelser = 10

    rskf = RepeatedStratifiedKFold(n_splits = splitt,
                                   n_repeats=gjentagelser,
                                   random_state=random_seed)

    rmse_validering_resultat = []

    for train, test in rskf.split(treningssett_markert,
                                  enzymtyper_trening_markert):

      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)

      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=300)

  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_enzym.loc[len(collagen_knr_resultater_enzym)] = resultater

i
i
i
i
j


In [33]:
collagen_knr_resultater_enzym

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.80,4.814298,0.395576,3.964583,0.194999,3.607101,0.515034,2.991426,0.145224,"{'n_neighbors': 8, 'p': 2}"
1,0.85,4.565014,0.456549,3.809012,0.184805,3.620759,0.511670,3.025672,0.147273,"{'n_neighbors': 9, 'p': 2}"
2,0.90,4.533221,0.464093,3.946889,0.189266,3.540743,0.518360,2.920236,0.141616,"{'n_neighbors': 10, 'p': 2}"
3,0.95,4.279386,0.522428,3.527284,0.170559,3.503121,0.522031,2.880385,0.139938,"{'n_neighbors': 9, 'p': 2}"
4,1.00,4.226460,0.534168,3.558778,0.169531,3.562287,0.515559,2.897913,0.141376,"{'n_neighbors': 10, 'p': 2}"
5,0.80,4.814298,0.395576,3.964583,0.194999,3.607101,0.515034,2.991426,0.145224,"{'n_neighbors': 8, 'p': 2}"
6,0.85,4.565014,0.456549,3.809012,0.184805,3.620759,0.511670,3.025672,0.147273,"{'n_neighbors': 9, 'p': 2}"
7,0.90,4.533221,0.464093,3.946889,0.189266,3.540743,0.518360,2.920236,0.141616,"{'n_neighbors': 10, 'p': 2}"
8,0.95,4.279386,0.522428,3.527284,0.170559,3.503121,0.522031,2.880385,0.139938,"{'n_neighbors': 9, 'p': 2}"
9,1.00,4.232501,0.532835,3.572424,0.170281,3.653331,0.490480,3.007284,0.147001,"{'n_neighbors': 11, 'p': 2}"


## Alternativ 2: Fordelt på dag og kontinuitet

### Alternativ 2: Et testsett

In [13]:
test_andel = 0.2
collagen_trening_markert, collagen_test,\
 _, _ = trening_testsett_oppdeling_dag(rå_data,
                                       collagen_data,
                                       test_andel=test_andel,
                                       trening_krav=True)

X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

**RandomForestRegressor**

Hyperparameteroptimalisering med Optuna

In [17]:
def objective(trial):
  parametere = {
      'n_estimators': trial.suggest_int('n_estimators', 100, 200),
      'max_depth': trial.suggest_int('max_depth', 1, 10)
  }
  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**parametere,
                                       random_state=random_seed))
  ])

  # Manuell Kryssvalidering med Kfold og optimalisering ift. gjennomsnittlig rmse

  splitt = 5

  treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                    collagen_trening_markert,
                                                                    n_splitt=splitt,
                                                                    trening_krav=True)
  rmse_validering_resultat = []

  for i, j in zip(treningssett, valideringssett):
    train = treningssett[i]
    val = valideringssett[j]

    X_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, :-1]
    X_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, :-1]

    y_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, -1]

    rfr_pipeline.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  rmse_validering_snitt = mean(rmse_validering_resultat)

  return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=100)

plot_optimization_history(study)

[I 2024-02-27 12:55:00,692] A new study created in memory with name: no-name-dead5878-7d35-45ec-afc1-c3bc6fb7eeae
[I 2024-02-27 12:55:02,972] Trial 0 finished with value: 3.200556663005775 and parameters: {'n_estimators': 172, 'max_depth': 10}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:04,351] Trial 1 finished with value: 3.5590960201980373 and parameters: {'n_estimators': 109, 'max_depth': 2}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:06,260] Trial 2 finished with value: 3.2397528638298247 and parameters: {'n_estimators': 180, 'max_depth': 7}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:08,127] Trial 3 finished with value: 3.5799845892864286 and parameters: {'n_estimators': 185, 'max_depth': 2}. Best is trial 0 with value: 3.200556663005775.
[I 2024-02-27 12:55:10,090] Trial 4 finished with value: 3.26589568073929 and parameters: {'n_estimators': 190, 'max_depth': 6}. Best is trial 0 with value: 3.200556663005775

In [18]:
# Bruker beste paramtere for studiet for å lage modell
beste_parametere = study.best_params

rfr_pipeline = Pipeline([
    ('skalerer', StandardScaler()),
    ('modell', RandomForestRegressor(**beste_parametere,
                                     random_state=random_seed))
])

# Trener modell for videre evaluering
rfr_pipeline.fit(X_trening_markert, y_trening_markert)
y_test_prediksjon = rfr_pipeline.predict(X_test)
y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_beregning(y_test,
                        y_test_prediksjon,
                        y_trening_markert,
                        y_trening_prediksjon)


# Legger resulater til i datasett
resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

In [19]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,2.814854,0.664399,2.59057,0.121022,1.305851,0.939135,1.058789,0.051623,"{'n_estimators': 120, 'max_depth': 10}"


### Alternativ 2: Flere testandeler

In [ ]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                      "RMSE test",
                                                      "R2 test",
                                                      "MAE test",
                                                      "MAPE test",
                                                      "RMSE trening",
                                                      "R2 trening",
                                                      "MAE trening",
                                                      "MAPE trening",
                                                      "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:
  collagen_trening, collagen_test, _, _ = trening_testsett_oppdeling_dag(rå_data,
                                                                         collagen_data,
                                                                         test_andel=test_andel)

  X_trening = collagen_trening.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]
  y_trening = collagen_trening.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  def objective(trial):
    parametere = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'max_depth': trial.suggest_int('max_depth', 1, 10)
    }
    rf_pipeline = Pipeline([
        ('skalerer', StandardScaler()),
        ('modell', RandomForestRegressor(**parametere, random_state=random_seed))
    ])

    rf_pipeline.fit(X_trening, y_trening)
    y_test_prediksjon = rf_pipeline.predict(X_test)
    rmse_test = rmse(y_test, y_test_prediksjon)
    return rmse_test

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere, random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                          y_test_prediksjon,
                          y_trening_markert,
                          y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

In [ ]:
collagen_rfr_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,1.193553,0.918419,1.091318,0.051979,1.218213,0.943346,0.999373,0.047705,"{'n_estimators': 180, 'max_depth': 7}"
1,0.2,1.126552,0.946246,0.935842,0.042563,1.173786,0.947403,0.939597,0.044500,"{'n_estimators': 105, 'max_depth': 10}"


## Altermativ 2: Flere treningssandeler

In [23]:
collagen_rfr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

collagen_knr_resultater_dag = pd.DataFrame(columns = ["Trenings andel",
                                                         "RMSE test",
                                                         "R2 test",
                                                         "MAE test",
                                                         "MAPE test",
                                                         "RMSE trening",
                                                         "R2 trening",
                                                         "MAE trening",
                                                         "MAPE trening",
                                                         "Beste parametere"])

test_andel = 0.2
collagen_trening_markert, collagen_test,\
collagen_trening_umarkert, collagen_trening = trening_testsett_oppdeling_dag(rå_data,
                                       collagen_data,
                                       test_andel=test_andel,
                                       trening_krav=True)

X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]
y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

In [24]:
# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       collagen_trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = collagen_trening_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # RandomForestRegressor
  def objective(trial):

    parametere = {
          'n_estimators': trial.suggest_int('n_estimators', 100, 200),
          'max_depth': trial.suggest_int('max_depth', 1, 10)
      }
    rfr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', RandomForestRegressor(**parametere,
                                           random_state=random_seed))
      ])

    splitt = 5

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      collagen_trening_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, -1]

      rfr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = rfr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  rfr_pipeline = Pipeline([
      ('skalerer', StandardScaler()),
      ('modell', RandomForestRegressor(**beste_parametere,
                                       random_state=random_seed))
  ])

  # Trener modell for videre evaluering
  rfr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = rfr_pipeline.predict(X_test)
  y_trening_prediksjon = rfr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_beregning(y_test,
                         y_test_prediksjon,
                         y_trening_markert,
                         y_trening_prediksjon)


  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_rfr_resultater_dag.loc[len(collagen_rfr_resultater_dag)] = resultater

[I 2024-02-27 13:15:22,861] A new study created in memory with name: no-name-37edc305-42f2-44b5-93fd-f044fbd2df63
[I 2024-02-27 13:15:24,376] Trial 0 finished with value: 3.2230325075200503 and parameters: {'n_estimators': 109, 'max_depth': 6}. Best is trial 0 with value: 3.2230325075200503.
[I 2024-02-27 13:15:27,884] Trial 1 finished with value: 3.236637952978445 and parameters: {'n_estimators': 188, 'max_depth': 9}. Best is trial 0 with value: 3.2230325075200503.
[I 2024-02-27 13:15:30,054] Trial 2 finished with value: 3.2301570168693754 and parameters: {'n_estimators': 153, 'max_depth': 4}. Best is trial 0 with value: 3.2230325075200503.
[I 2024-02-27 13:15:31,567] Trial 3 finished with value: 3.203982991301071 and parameters: {'n_estimators': 108, 'max_depth': 8}. Best is trial 3 with value: 3.203982991301071.
[I 2024-02-27 13:15:33,162] Trial 4 finished with value: 3.2799154660204546 and parameters: {'n_estimators': 121, 'max_depth': 3}. Best is trial 3 with value: 3.203982991301

In [25]:
collagen_rfr_resultater_dag

,Trenings andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.8,3.244672,0.554084,2.975020,0.136084,1.434596,0.926134,1.105496,0.055041,"{'n_estimators': 120, 'max_depth': 10}"
1,0.9,2.852612,0.655335,2.581855,0.119024,1.647095,0.904888,1.368819,0.066429,"{'n_estimators': 123, 'max_depth': 5}"
2,1.0,2.811634,0.665166,2.581306,0.121389,1.581324,0.910747,1.325922,0.063735,"{'n_estimators': 101, 'max_depth': 5}"


In [27]:
# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       collagen_trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = collagen_trening_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening

  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  # KneigborsRegressor
  def objective(trial):

    parametere = {
          'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
          'p': trial.suggest_int('p', 1, 20)
      }
    knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**parametere))
      ])


    splitt = 5

    treningssett, valideringssett = trening_validering_oppdeling_dag(rå_data,
                                                                      collagen_trening_markert,
                                                                      n_splitt=splitt,
                                                                      trening_krav=True)
    rmse_validering_resultat = []

    for i, j in zip(treningssett, valideringssett):
      train = treningssett[i]
      val = valideringssett[j]

      X_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, :-1]
      X_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, :-1]

      y_trening_fold = (collagen_trening_markert.loc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.loc[val, :]).iloc[:, -1]

      knr_pipeline.fit(X_trening_fold, y_trening_fold)

      y_validering_prediksjon = knr_pipeline.predict(X_validering_fold)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt


  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)


  # Bruker beste paramtere for studiet for å lage modell
  beste_parametere = study.best_params

  knr_pipeline = Pipeline([
          ('skalerer', StandardScaler()),
          ('modell', KNeighborsRegressor(**beste_parametere))
      ])

  # Trener modell for videre evaluering
  knr_pipeline.fit(X_trening_markert, y_trening_markert)
  y_test_prediksjon = knr_pipeline.predict(X_test)
  y_trening_prediksjon = knr_pipeline.predict(X_trening_markert)

  # Henter ut evalueringsresulateter av beste modell fra undersøkelse
  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                   y_test_prediksjon,
                                                   y_trening_markert,
                                                   y_trening_prediksjon)

  # Legger resulater til i datasett
  resultater = [trenings_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_knr_resultater_dag.loc[len(collagen_knr_resultater_dag)] = resultater

[I 2024-02-27 13:36:04,802] A new study created in memory with name: no-name-dc0dfffd-4816-4e23-803b-e3b581a11ce4
[I 2024-02-27 13:36:05,542] Trial 0 finished with value: 3.6349327092463093 and parameters: {'n_neighbors': 8, 'p': 6}. Best is trial 0 with value: 3.6349327092463093.
[I 2024-02-27 13:36:06,506] Trial 1 finished with value: 3.57259149229613 and parameters: {'n_neighbors': 7, 'p': 18}. Best is trial 1 with value: 3.57259149229613.
[I 2024-02-27 13:36:07,821] Trial 2 finished with value: 4.064071406223955 and parameters: {'n_neighbors': 14, 'p': 12}. Best is trial 1 with value: 3.57259149229613.
[I 2024-02-27 13:36:08,836] Trial 3 finished with value: 3.097053667420894 and parameters: {'n_neighbors': 2, 'p': 11}. Best is trial 3 with value: 3.097053667420894.
[I 2024-02-27 13:36:09,848] Trial 4 finished with value: 3.0716211033199032 and parameters: {'n_neighbors': 2, 'p': 18}. Best is trial 4 with value: 3.0716211033199032.
[I 2024-02-27 13:36:10,806] Trial 5 finished with 

# NIR målinger ekskludert